<a href="https://colab.research.google.com/github/maheshboj/agenticai_basics/blob/Langchain_components/Building_an_Agentic_Corrective_RAG_System_with_LangGraph_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Building an Agentic Corrective RAG System with LangGraph

This project will cover a full hands-on workflow and demonstration of how to build an Agentic Corrective RAG (CRAG) System with LangGraph

The idea would be to implement the workflow taking inspiration from the [Corrective Retrieval Augmented Generation](https://arxiv.org/pdf/2401.15884) research paper.

The main challenge of RAG systems include:

- Poor Retrieval can lead to issues in LLM response generation
- Bad retrieval or lack of information in the vector database can also lead to out of context or hallucinated answers

The idea is to couple a RAG system with a few checks in place and perform web searches if there is a lack of relevant context documents to the given user query as follows:

![](https://i.imgur.com/uhybMhT.png)


We can build this as an agentic RAG system by having a specific functionality step as a node in the graph and use LangGraph to implement it. Key steps in the node will include prompts being sent to LLMs to perform specific tasks as seen in the detailed workflow below:

![](https://i.imgur.com/eV87ZwX.gif)

## Install OpenAI, and LangChain dependencies

Install the following httpx library version for compatibility with other libraries

In [11]:
%pip install -qU httpx langchain langchain-openai langgraph langchain-tavily langchain_community langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.5 MB/s eta 0:00:00


In [111]:
from google.colab import userdata


## Enter Open AI API Key

In [109]:
OPENAI_KEY = userdata.get('OPENAI_APIKEY')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [110]:
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

## Setup Environment Variables

In [113]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Build a Search Index for Wikipedia Data

We will build a vector database for retrieval and search by taking a subset of documents from wikipedia, similar to our project from previous modules

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [114]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

### Get the wikipedia data

In [115]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW download it
# manually upload it on colab
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=d5c56dd0-e950-46c6-8c1d-0f8bce079eb2
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:01<00:00, 27.8MB/s]


### Load and Chunk Documents

In [116]:
import gzip
import json
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        #Add all paragraphs
        #passages.extend(data['paragraphs'])
        #Only add the first paragraph
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3]) # restrict data to first 3 paragraphs to run later modules faster
        })

# We subset our data so we only use a subset of wikipedia documents to run things faster
docs = [doc for doc in docs for x in ['india']
              if x in doc['data'].lower().split()]
# Create docs
docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]
# Chunk docs
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [117]:
len(chunked_docs)

1322

In [118]:
chunked_docs[:3]

[Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content='Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silky leaves 3–5\xa0cm long and 1–3\xa0cm broad. The leaves are opposite each other. The flowers are quite big. They are white in color and arranged as a spike. The plant tastes somewhat like anise, with a strong, pungent, sweet smell. Basil is very sensitive to cold. It is best grown in hot, dry conditions. While most common varieties are treated as annuals, some are perennial, including African Blue a

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [119]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='rag_wikipedia_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wikipedia_db")

### Setup a Vector Database Retriever

Here we use the following retrieval strategy:

- Similarity with Threshold Retrieval


### Similarity with Threshold Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query and also introduce a cutoff to not return any documents which are below a certain similarity threshold

In [120]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                        search_kwargs={"k": 3,
                                                                       "score_threshold": 0.3})

In [121]:
query = "how far is mysore from bangalore?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(id='e8b56560-73c9-4e0f-85b8-cb0f5b2d9bae', metadata={'article_id': '603003', 'title': 'Second Anglo-Mysore War'}, page_content='The Second Anglo-Mysore War was a conflict that took place on the Indian Subcontinent from 1780 to 1784. The war ended without a clear victory on either side. The Treaty of Mangalore restored the situation to the state before the war broke out. The war involved the Kingdom of Mysore and the British East India Company. The Kingdom of Mysore was a large kingdom, in the South of India. It was an ally of the French. The East India Company was on the side of the English. Hyder Ali, the ruler of Mysore, committed himself to a French alliance to seek revenge against the British. The British had problems with Mysore because it had an alliance with the French. The army of Hyder was one of the largest armies in India. In 1780, his army swept down the Eastern Ghats (mountains) and burnt the villages. Hyder set about forming a confederacy against the British. Th

In [122]:
query = "what is langgraph?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[]

## Create a Query Retrieval Grader

Here we will use an LLM itself to grade if any retrieved document is relevant to the given question - Answer will be either `yes` or `no`

In [123]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model for LLM output format
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM for grading
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt template for grading
SYS_PROMPT = """You are an expert grader assessing relevance of a retrieved document to a user question.
                Follow these instructions for grading:
                  - If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
                  - Your grade should be either 'yes' or 'no' to indicate whether the document is relevant to the question or not.
             """
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Retrieved document:
                     {document}

                     User question:
                     {question}
                  """),
    ]
)

# Build grader chain
doc_grader = (grade_prompt
                  |
              structured_llm_grader)

In [124]:
query = "where is mysore?"
top3_docs = similarity_threshold_retriever.invoke(query)
for doc in top3_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

Tipu Sultan (20 November 1750 – 4 May 1799), also known as the Tiger of Mysore, was the ruler of the Kingdom of Mysore from 1782 to 1799. He was also a scholar, soldier and poet. Tipu was the eldest son of Sultan Hyder Ali of Mysore and his wife Fatima Fakhr-un-Nisa. Tipu Sultan born in a Muslim family. His ancestors are said by different sources to be from Persia, Afghanistan, Arabia, and Ferghana in present day Uzbekistan. Tipu introduced several new laws, including new coinage, a new lunisolar calendar and a new land revenue system. He started the growth of the silk industry in Mysore. Tipu inherited a large kingdom bordered by the Krishna River in the south, the Eastern Ghats in the east and the Arabian Sea in the west. At the request of the French, he built a church, the first in Mysore. With French help, Tipu Sultan fought against the British to keep Mysore's independence. Tipu Sultan, prominently known as Sher-e-Mysore (Tiger of Mysore) is also given the sobriquet of Sher-e-Hind

In [125]:
query = "who is tippu sultan?"
top3_docs = similarity_threshold_retriever.invoke(query)
for doc in top3_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

Tipu Sultan (20 November 1750 – 4 May 1799), also known as the Tiger of Mysore, was the ruler of the Kingdom of Mysore from 1782 to 1799. He was also a scholar, soldier and poet. Tipu was the eldest son of Sultan Hyder Ali of Mysore and his wife Fatima Fakhr-un-Nisa. Tipu Sultan born in a Muslim family. His ancestors are said by different sources to be from Persia, Afghanistan, Arabia, and Ferghana in present day Uzbekistan. Tipu introduced several new laws, including new coinage, a new lunisolar calendar and a new land revenue system. He started the growth of the silk industry in Mysore. Tipu inherited a large kingdom bordered by the Krishna River in the south, the Eastern Ghats in the east and the Arabian Sea in the west. At the request of the French, he built a church, the first in Mysore. With French help, Tipu Sultan fought against the British to keep Mysore's independence. Tipu Sultan, prominently known as Sher-e-Mysore (Tiger of Mysore) is also given the sobriquet of Sher-e-Hind

In [126]:
query = "who won the champions league in 2024?"
top3_docs = similarity_threshold_retriever.invoke(query)
for doc in top3_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

The 2023 ICC Cricket World Cup is scheduled to be hosted by India and India was selected as the host at an International Cricket Council (ICC) meeting in London in June 2013. This will be the 13th Cricket World Cup competition. It will be the fourth time that India will be the host. This will be the first time that India has hosted the tournament on its own. India hosted previous World Cup tournaments in 1987 (with Pakistan), 1996 (with Pakistan and Sri Lanka) and 2011 (with Sri Lanka and Bangladesh). The semi final will be played at Wankhede Stadium. And final will be played at Eden Gardens, Kolkata. 5. ICC Cricket Worldcup 2023 Venue
GRADE: binary_score='no'

The 2023 ICC Cricket World Cup is scheduled to be hosted by India and India was selected as the host at an International Cricket Council (ICC) meeting in London in June 2013. This will be the 13th Cricket World Cup competition. It will be the fourth time that India will be the host. This will be the first time that India has hos

## Build a QA RAG Chain

We will now connect our retriever to an LLM and build our QA RAG Chain

In [127]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know the answer.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer and to the point answer with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """
prompt_template = ChatPromptTemplate.from_template(prompt)

chatgpt = ChatOpenAI(model_name='gpt-4o', temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter('question')
    }
      |
    prompt_template
      |
    chatgpt
      |
    StrOutputParser()
)

In [128]:
query = "where is mysore?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top3_docs, "question": query}
)
print(result)

The context provided does not explicitly state where Mysore is located. However, based on general knowledge, Mysore is a city in the southern part of India, in the state of Karnataka. It is known for its rich cultural heritage and historical significance, particularly during the reign of Tipu Sultan, who was a prominent ruler of the Kingdom of Mysore.


In [129]:
query = "who won the champions league in 2024?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top3_docs, "question": query}
)
print(result)

I don't know the answer. The provided context does not contain information about the winner of the Champions League in 2024.


## Create a Query Rephraser

We will now build a query rephraser which will use an LLM to rephrase the input user query into a better version which is optimized for web search

In [130]:
# LLM for question rewriting
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompt template for rewriting
SYS_PROMPT = """Act as a question re-writer and perform the following task:
                 - Convert the following input question to a better version that is optimized for web search.
                 - When re-writing, look at the input question and try to reason about the underlying semantic intent / meaning.
             """
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Here is the initial question:
                     {question}

                     Formulate an improved question.
                  """,
        ),
    ]
)

question_rewriter = (re_write_prompt
                        |
                       llm
                        |
                     StrOutputParser())

In [131]:
query = "palces to visit in mysore?"
question_rewriter.invoke({"question": query})

'What are the top tourist attractions to visit in Mysore?'

## Load Web Search Tool

Here we will be using the [Tavily API](https://tavily.com/#api) for our web searches

In [132]:
from google.colab import userdata
import os

In [133]:
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [134]:
from langchain_tavily import TavilySearch
tv_search = TavilySearch(
    max_results=5,              # Default: 5 results
    search_depth="advanced",    # "basic" or "advanced"
    include_answer=True,        # Include summarized answer
    include_raw_content=False,  # Include full page content
    include_images=False,       # Include image results
    topic="general",             # "general", "news", "finance"

    )

In [135]:


# Call directly
results = tv_search.invoke({'query': 'what is the distance between bangalore and mysore?'})
print(results)

{'query': 'what is the distance between bangalore and mysore?', 'follow_up_questions': None, 'answer': 'The distance between Bangalore and Mysore is approximately 145 km. The fastest route is via NH275, taking about 3 hours. Trains are a common and efficient mode of transport.', 'images': [], 'results': [{'url': 'https://www.tejastravels.com/blogs/travel-guides/bangalore-to-mysore-distance-travel-guide', 'title': 'Bangalore to Mysore Distance: Best Routes, Travel Time & ...', 'content': "## How Far is Bangalore from Mysore?\n\nWondering how far Bangalore is from Mysore? The distance is about 145 km, making it a quick and easy trip. Consider Tejas Travels for a comfortable, hassle-free journey. Book your package today!\n\nDistance between Bangalore and Mysore\n\nThe distance between Bangalore and Mysore is approximately 143 km. You can travel for work or leisure with Tejas Travels’ hassle-free packages. It will make your journey comfortable.\n\nTravel time by different modes [...] An al

In [136]:
results['results']

[{'url': 'https://www.tejastravels.com/blogs/travel-guides/bangalore-to-mysore-distance-travel-guide',
  'title': 'Bangalore to Mysore Distance: Best Routes, Travel Time & ...',
  'content': "## How Far is Bangalore from Mysore?\n\nWondering how far Bangalore is from Mysore? The distance is about 145 km, making it a quick and easy trip. Consider Tejas Travels for a comfortable, hassle-free journey. Book your package today!\n\nDistance between Bangalore and Mysore\n\nThe distance between Bangalore and Mysore is approximately 143 km. You can travel for work or leisure with Tejas Travels’ hassle-free packages. It will make your journey comfortable.\n\nTravel time by different modes [...] An alternative route from Bangalore to Mysore is NH948 via Kanakapura Road. This route offers less traffic and a more scenic drive. It is perfect for a relaxed trip. While it may take a little longer, yet it’s a peaceful experience.\n\nRoute comparison\n\nYou can choose between NH948 and NH275 for your tr

In [137]:
results['results'][0]['content']

"## How Far is Bangalore from Mysore?\n\nWondering how far Bangalore is from Mysore? The distance is about 145 km, making it a quick and easy trip. Consider Tejas Travels for a comfortable, hassle-free journey. Book your package today!\n\nDistance between Bangalore and Mysore\n\nThe distance between Bangalore and Mysore is approximately 143 km. You can travel for work or leisure with Tejas Travels’ hassle-free packages. It will make your journey comfortable.\n\nTravel time by different modes [...] An alternative route from Bangalore to Mysore is NH948 via Kanakapura Road. This route offers less traffic and a more scenic drive. It is perfect for a relaxed trip. While it may take a little longer, yet it’s a peaceful experience.\n\nRoute comparison\n\nYou can choose between NH948 and NH275 for your trip from Bangalore to Mysore. You can consider the following points:\n\nCriteria\n\nNH275 (Bangalore-Mysore Expressway)\n\nNH948 (Kanakapura Road)\n\nDistance\n\n19.2 km\n\n36.9 km\n\nRoad con

In [138]:
concatenated_content = "\n\n".join([d['content'] for d in results['results']])
print(concatenated_content)

## How Far is Bangalore from Mysore?

Wondering how far Bangalore is from Mysore? The distance is about 145 km, making it a quick and easy trip. Consider Tejas Travels for a comfortable, hassle-free journey. Book your package today!

Distance between Bangalore and Mysore

The distance between Bangalore and Mysore is approximately 143 km. You can travel for work or leisure with Tejas Travels’ hassle-free packages. It will make your journey comfortable.

Travel time by different modes [...] An alternative route from Bangalore to Mysore is NH948 via Kanakapura Road. This route offers less traffic and a more scenic drive. It is perfect for a relaxed trip. While it may take a little longer, yet it’s a peaceful experience.

Route comparison

You can choose between NH948 and NH275 for your trip from Bangalore to Mysore. You can consider the following points:

Criteria

NH275 (Bangalore-Mysore Expressway)

NH948 (Kanakapura Road)

Distance

19.2 km

36.9 km

Road conditions [...] Two of the mo

## Build Agentic RAG components

Here we will build the key components of our Agentic Corrective RAG System as per the workflow below:

![](https://i.imgur.com/uhybMhT.png)



### Graph State

Used to store and represent the state of the agent graph as we traverse through various nodes

In [139]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM response generation
        web_search_needed: flag of whether to add web search - yes or no
        documents: list of context documents
    """

    question: str
    generation: str
    web_search_needed: str
    documents: List[str]

### Retrieve function for retrieval from Vector DB

This will be used to get relevant context documents from the vector database

In [140]:
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents - that contains retrieved context documents
    """
    print("---RETRIEVAL FROM VECTOR DB---")
    question = state["question"]

    # Retrieval
    documents = similarity_threshold_retriever.invoke(question)
    return {"documents": documents, "question": question}

### Grade documents

This will be used to determine whether the retrieved documents are relevant to the question by using an LLM Grader

Sets the `web_search_needed` flag as `Yes` if at least one document is not contextually relevant and sets it as `No` if all documents are contextually relevant to the given user query

In [141]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    by using an LLM Grader.

    If any document are not relevant to question or documents are empty - Web Search needs to be done
    If all documents are relevant to question - Web Search is not needed
    Helps filtering out irrelevant documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search_needed = "No"
    if documents:
        for d in documents:
            score = doc_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            grade = score.binary_score
            if grade == "yes":
                print("---GRADE: DOCUMENT RELEVANT---")
                filtered_docs.append(d)
            else:
                print("---GRADE: DOCUMENT NOT RELEVANT---")
                web_search_needed = "Yes"
                continue
    else:
        print("---NO DOCUMENTS RETRIEVED---")
        web_search_needed = "Yes"

    return {"documents": filtered_docs, "question": question, "web_search_needed": web_search_needed}

### Rewrite query

This will be used to rewrite the input query to produce a better question optimized for web search using an LLM

In [142]:
def rewrite_query(state):
    """
    Rewrite the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased or re-written question
    """

    print("---REWRITE QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

### Web Search

This will be used to search the web using the web search tool for the given query and retrieve some information which can be used as the context in RAG

In [143]:
from langchain_core.documents import Document

def web_search(state):
    """
    Web search based on the re-written question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = tv_search.invoke({'query': question})
    docs = tv_search.invoke({'query': question})
    web_results = "\n\n".join([d['content'] for d in docs['results']])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}

### Generate Answer

Standard LLM Response generation from query and context documents in a RAG system

In [144]:
def generate_answer(state):
    """
    Generate answer from context document using LLM

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = qa_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

### Decide to Generate

This will be used as a conditional function which will check the `web_search_needed` flag and decide if a web search is needed or a response should be generated and return the function name to be called

In [145]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    web_search_needed = state["web_search_needed"]

    if web_search_needed == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: SOME or ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, REWRITE QUERY---")
        return "rewrite_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE RESPONSE---")
        return "generate_answer"

### Build the Agent Graph

Here we will use LangGraph and build the agent as a graph

In [146]:
from langgraph.graph import END,StateGraph

agentic_rag = StateGraph(GraphState)

# Define the nodes
agentic_rag.add_node("retrieve", retrieve)  # retrieve
agentic_rag.add_node("grade_documents", grade_documents)  # grade documents
agentic_rag.add_node("rewrite_query", rewrite_query)  # transform_query
agentic_rag.add_node("web_search", web_search)  # web search
agentic_rag.add_node("generate_answer", generate_answer)  # generate answer

# Build graph
agentic_rag.set_entry_point("retrieve")
agentic_rag.add_edge("retrieve", "grade_documents")
agentic_rag.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {"rewrite_query": "rewrite_query", "generate_answer": "generate_answer"},
)
agentic_rag.add_edge("rewrite_query", "web_search")
agentic_rag.add_edge("web_search", "generate_answer")
agentic_rag.add_edge("generate_answer", END)

# Compile
agentic_rag = agentic_rag.compile()

In [147]:
# from IPython.display import Image, display, Markdown
# from langchain_core.runnables.graph import MermaidDrawMethod
# import nest_asyncio
# nest_asyncio.apply()

# !pip install pyppeteer
# #
# display(Image(agentic_rag.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

## Test the Agentic CRAG System

In [148]:
query = "what is the capital of India?"
response = agentic_rag.invoke({"question": query})

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE RESPONSE---
---GENERATE ANSWER---


In [149]:
display(Markdown(response['generation']))

The capital of India is New Delhi. It is also a union territory within the larger megacity of Delhi.

In [150]:
response

{'question': 'what is the capital of India?',
 'generation': 'The capital of India is New Delhi. It is also a union territory within the larger megacity of Delhi.',
 'web_search_needed': 'No',
 'documents': [Document(id='120cc756-2264-4e0a-8afe-d642bfaac52c', metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
  Document(id='264a5c0d-4097-40d4-aeb9-5db0952c9be8', metadata={'title': 'New Delhi', 'article_id': '5117'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In tradit

In [151]:
query = "who won the champions league in 2024?"
response = agentic_rag.invoke({"question": query})

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: SOME or ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
---GENERATE ANSWER---


In [152]:
display(Markdown(response['generation']))

The winner of the 2024 UEFA Champions League was Real Madrid. They defeated Borussia Dortmund 2-0 in the final held at Wembley Stadium in London on June 1, 2024.

In [153]:
query = "Tell me about India"
response = agentic_rag.invoke({"question": query})

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE RESPONSE---
---GENERATE ANSWER---


In [154]:
display(Markdown(response['generation']))

India is a country located in Asia, specifically at the center of South Asia. It is the seventh largest country in the world by area and the largest in South Asia. With a population exceeding 1.2 billion people, India is the second most populous country globally and holds the distinction of being the most populous democracy in the world.

Geographically, India is a peninsula, bordered by the Indian Ocean to the south, the Arabian Sea to the west, and the Bay of Bengal to the east. It shares its borders with seven countries: Pakistan to the northwest, China and Nepal to the north, Bhutan and Bangladesh to the northeast, Myanmar to the east, and Sri Lanka, an island nation, to the south.

The capital city of India is New Delhi. The country has a significant military presence, being home to the third largest military force in the world, and it is recognized as a nuclear weapon state.

In [155]:
query = "How far is mysore from bangalore"
response = agentic_rag.invoke({"question": query})

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: SOME or ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
---GENERATE ANSWER---


In [156]:
display(Markdown(response['generation']))

The distance between Bangalore and Mysore varies slightly depending on the route taken, but it is generally around 140 to 150 kilometers by road. The most commonly cited distance is approximately 145 kilometers. The travel time by car can range from 1.5 to 2 hours, especially when using the Bengaluru–Mysuru Expressway (NH275), which is a modern, six-lane, access-controlled highway. The expressway helps reduce travel time significantly, making the journey quicker and more comfortable.